## Read Text Data From File
<ul>
<li> remove label 'handle', 'tweet'</li>
    <li> encoding = 'utf-8'</li>
    <li> labelList:  Clinton: 1 ; Trump: 0</li>
    <li> print size</li>
</ul>

In [16]:
## read text data from file

import csv
import string

labelList = []
tweetList = []

with open('train.csv', encoding='utf-8') as trainfile:
    reader = csv.reader(trainfile)
    for row in reader:
        if row[0] == 'HillaryClinton':
            labelList.append(1)
        elif row[0] == 'realDonaldTrump':
            labelList.append(0)
            
        if row[1] != 'tweet':
            tweetList.append(row[1])

print(len(tweetList))
print(len(labelList))

4743
4743


## Create Corpus
<ul>
    <li>unique word in corpus</li>
</ul>

In [17]:
## corpus

corpus = set()

for tweet in tweetList:
    words = tweet.split();
    corpus.update(words)

corpus = list(corpus)
N = len(corpus)
print(N)

18337


## Label Encoding

<ul>
    <li>hash table encoder each word to unique number</li>
    <li>encode entire tweet list</li>
    <li>decode method</li>
</ul>

In [18]:
## Label Encoding

hashEncoder = {}
i = 0
for word in corpus:
    hashEncoder[word] = i
    i += 1
    
tweetList_encoded = []
for tweet in tweetList:
    tweet_encoded = []
    for word in tweet.split():
        word_encoded = hashEncoder[word]
        tweet_encoded.append(word_encoded)
    tweetList_encoded.append(tweet_encoded)

def decodeTweet(tweet_encoded):
    decodeList = []
    for word_encoded in tweetList_encoded[5]:
        decodeList.append(list(hashEncoder.keys())[list(hashEncoder.values()).index(word_encoded)])
    return decodeList
    
print(len(tweetList_encoded))
print(tweetList[5])
print(tweetList_encoded[5])
print(decodeTweet(tweetList_encoded[5]))

4743
When Donald Trump goes low...register to vote: https://t.co/tTgeqxNqYm https://t.co/DXz9dEwsZS
[10824, 8499, 2881, 3322, 15921, 4991, 17210, 11427, 11861]
['When', 'Donald', 'Trump', 'goes', 'low...register', 'to', 'vote:', 'https://t.co/tTgeqxNqYm', 'https://t.co/DXz9dEwsZS']


In [2]:
import tensorflow as tf

In [ ]:
## One Hot Matrix
oneHotMatrix = tf.eye(N, name='oneHotMatrix')